In [1]:
import pandas as pd

df=pd.read_csv('fake_news.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.shape

(606, 5)

In [3]:
df=df.dropna()
df.shape

(606, 5)

In [4]:
infos=df.copy()
infos.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [5]:
#the independent features

X=infos.drop('label',axis=1)

In [6]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
#the dependent feature

y=infos['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [8]:
#lowercase
#stemming
#removing stopwords
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(infos)):
    final = re.sub('[^a-zA-Z]', ' ', infos['text'][i])
    final = final.lower()
    final = final.split()
    
    final = [ps.stem(word) for word in final if not word in stopwords.words('english')]
    final = ' '.join(final)
    corpus.append(final)

In [9]:
#applying Countvectorizer
#creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=80,ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [10]:
X.shape

(606, 80)

In [11]:
y=infos['label']

In [12]:
#train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [13]:
cv.get_feature_names()[:10]

['also',
 'american',
 'back',
 'call',
 'campaign',
 'clinton',
 'come',
 'could',
 'countri',
 'day']

In [14]:
dataset = pd.DataFrame(X_train, columns=cv.get_feature_names())
dataset.head()

,also,american,back,call,campaign,clinton,come,could,countri,day,...,vote,want,war,way,well,white,work,world,would,year
0,0,0,0,4,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6,0,0,5,0,1,0,0,1,2,...,0,2,0,1,0,3,2,2,1,2
2,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,3
3,6,17,4,5,1,0,4,4,3,16,...,0,13,1,10,1,2,75,2,4,20
4,1,1,2,0,3,3,0,1,1,1,...,0,0,5,0,0,2,0,1,1,1


# PassiveAggressive Classifier

In [15]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics

linear_clf = PassiveAggressiveClassifier(max_iter=30, random_state=1)
linear_clf.fit(X_train, y_train)
prediction = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test, prediction)
print("accuracy:", score)

accuracy: 0.7786885245901639


In [16]:
#features names
feature_names = cv.get_feature_names()
#most real words
sorted(zip(linear_clf.coef_[0], feature_names), reverse=True)[:20]

[(1.0532252610781692, 'elect'),
 (0.7218382449189956, 'war'),
 (0.57305483943563, 'la'),
 (0.5488681992984451, 'call'),
 (0.5338900023924266, 'take'),
 (0.5229408816976022, 'use'),
 (0.5068984233142831, 'obama'),
 (0.4535931292278683, 'well'),
 (0.4353381879494659, 'republican'),
 (0.4296728940000136, 'us'),
 (0.412054398243737, 'thing'),
 (0.4092140947218282, 'nation'),
 (0.3892909216772175, 'clinton'),
 (0.3702781582525995, 'year'),
 (0.3544475560001156, 'help'),
 (0.35008563711415996, 'hillari'),
 (0.2978887006141646, 'million'),
 (0.2859304363141996, 'like'),
 (0.2643432966952469, 'need'),
 (0.2579520274975033, 'campaign')]

In [17]:
#most fake words
sorted(zip(linear_clf.coef_[0], feature_names))[:20]

[(-0.8135583837215248, 'say'),
 (-0.7390473931058924, 'mr'),
 (-0.6705241479817761, 'countri'),
 (-0.6364062093268772, 'part'),
 (-0.4588133609783679, 'said'),
 (-0.4577723543400703, 'presid'),
 (-0.37905952385932234, 'unit state'),
 (-0.3511407895963263, 'tri'),
 (-0.34119206347420644, 'de'),
 (-0.33873869551817687, 'could'),
 (-0.3335728382297425, 'way'),
 (-0.33242493685019325, 'live'),
 (-0.329322583044493, 'two'),
 (-0.3101681831019795, 'mr trump'),
 (-0.291370495251231, 'much'),
 (-0.26973773097946263, 'may'),
 (-0.24769667606658619, 'group'),
 (-0.24547790138579426, 'state'),
 (-0.23124923431395158, 'hous'),
 (-0.21428864958940694, 'look')]

# SGD Classifier

In [18]:
from sklearn.linear_model import SGDClassifier

text_clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=30,random_state=1)
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, predicted)
print("accuracy:", score)

accuracy: 0.7868852459016393


In [19]:
#most real words
sorted(zip(text_clf.coef_[0], feature_names), reverse=True)[:20]

[(158.02733033055813, 'la'),
 (82.44904191159574, 'us'),
 (80.1587907473848, 'elect'),
 (70.23436903580405, 'help'),
 (64.12703259790777, 'hillari'),
 (53.4391938315899, 'govern'),
 (51.91235972211605, 'clinton'),
 (46.56844033895693, 'well'),
 (46.56844033895671, 'right'),
 (41.22452095579789, 'world'),
 (38.93426979158697, 'vote'),
 (35.88060157263885, 'use'),
 (31.300099244216955, 'one'),
 (29.773265134742942, 'includ'),
 (27.48301397053198, 'white'),
 (25.192762806321028, 'campaign'),
 (24.429345751583867, 'thing'),
 (22.90251164210992, 'take'),
 (21.375677532636097, 'obama'),
 (19.848843423162005, 'may')]

In [20]:
#most fake words
sorted(zip(text_clf.coef_[0], feature_names))[:20]

[(-75.57828841896297, 'way'),
 (-64.127032597908, 'much'),
 (-63.36361554317096, 'unit state'),
 (-50.385525612641665, 'mr'),
 (-48.85869150316801, 'even'),
 (-48.09527444843101, 'mr trump'),
 (-46.56844033895705, 'polit'),
 (-43.514772120008914, 'offic'),
 (-40.46110390106094, 'last'),
 (-35.11718451790196, 'made'),
 (-33.59035040842797, 'say'),
 (-32.826933353691, 'go'),
 (-29.009848080005966, 'first'),
 (-28.246431025268826, 'said'),
 (-27.48301397053198, 'new'),
 (-24.429345751584123, 'hous'),
 (-20.612260477899053, 'mani'),
 (-19.848843423162077, 'day'),
 (-19.08542636842588, 'de'),
 (-17.55859225895095, 'show')]

# Tf-idf scoring 

In [21]:
# Construction of the tf-idf score matrix
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X)
X_train_tfidf.shape

(606, 80)

In [22]:
from sklearn.pipeline import Pipeline

text_clf_2 = Pipeline([('tf-idf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',max_iter=50, random_state=1))])
text_clf_2 = text_clf_2.fit(X_train, y_train)
predicted_2 = text_clf_2.predict(X_test)
score_2 = metrics.accuracy_score(y_test, predicted_2)
print("accuracy:", score_2)

accuracy: 0.7950819672131147
